In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import requests
import pandas as pd
from io import StringIO
from pathlib import Path
import json


def load_github_csv(url):
    response = requests.get(url)
    response.raise_for_status()
    df = pd.read_csv(StringIO(response.text))
    return df


df = load_github_csv(
    "https://raw.githubusercontent.com/dfhahn/protein-ligand-benchmark-analysis/master/03_comparison_experiment/03b_all_ligands_all_ffs.csv"
)

In [2]:
df = df[["ligand", "target", "DG_Exp.", "dDG_Exp."]]

In [17]:
lomap_jsons = sorted(Path("../perturbations").glob("*/lomap.json"))
for json_file in lomap_jsons:

    results_dict = json.loads(json_file.read_text())
    target = json_file.parent.name
    print(f"Processing: {target}")
    for key, node in results_dict["nodes"].items():
        subset = df.query("target == @target & ligand == @key")
        assert (
            subset.shape[0] == 1
        ), f"Something went wrong for target {target}, ligand {key}"
        dg = subset["DG_Exp."].values[0]
        dg_err = subset["dDG_Exp."].values[0]
        node.update({"dg_value": dg, "dg_error": dg_err})

    json_file.write_text(json.dumps(results_dict, indent=4))

Processing: bace
Processing: bace_hunt
Processing: bace_p2
Processing: cdk2
Processing: cdk8
Processing: cmet
Processing: eg5
Processing: galectin
Processing: hif2a
Processing: jnk1
Processing: mcl1
Processing: p38
Processing: pde10
Processing: pde2
Processing: pfkfb3
Processing: ptp1b
Processing: shp2
Processing: syk
Processing: thrombin
Processing: tnks2
Processing: tyk2
